In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.SILENT)

# Clear variables (equivalent to MATLAB's clear solver AvgTable)
solver = None
AvgTable = None

In [2]:
# MATLAB: model = LayeredNetwork('LQN1');
model = LayeredNetwork('LQN1')

# definition of processors, tasks and entries
# MATLAB: P1 = Processor(model, 'P1', Inf, SchedStrategy.INF);
# MATLAB: T1 = Task(model, 'T1', 1, SchedStrategy.REF).on(P1);
# MATLAB: E1 = Entry(model, 'E1').on(T1);
P1 = Processor(model, 'P1', float('inf'), SchedStrategy.INF)
T1 = Task(model, 'T1', 1, SchedStrategy.REF).on(P1)
E1 = Entry(model, 'E1').on(T1)

# MATLAB: P2 = Processor(model, 'P2', Inf, SchedStrategy.INF);
# MATLAB: T2 = Task(model, 'T2', Inf, SchedStrategy.INF).on(P2);
# MATLAB: E2 = Entry(model, 'E2').on(T2);
P2 = Processor(model, 'P2', float('inf'), SchedStrategy.INF)
T2 = Task(model, 'T2', float('inf'), SchedStrategy.INF).on(P2)
E2 = Entry(model, 'E2').on(T2)

# definition of activities
# MATLAB: T1.setThinkTime(Erlang.fitMeanAndOrder(0.0001,2));
T1.setThinkTime(Erlang.fitMeanAndOrder(0.0001, 2))

# MATLAB: A1 = Activity(model, 'A1', Exp(1.0)).on(T1).boundTo(E1).synchCall(E2,3);
# MATLAB: A2 = Activity(model, 'A2', APH.fitMeanAndSCV(1,10)).on(T2).boundTo(E2).repliesTo(E2);
A1 = Activity(model, 'A1', Exp(1.0)).on(T1).boundTo(E1).synchCall(E2, 3)
A2 = Activity(model, 'A2', APH.fitMeanAndSCV(1, 10)).on(T2).boundTo(E2).repliesTo(E2)

In [3]:
# instantiate solvers
# MATLAB: options = SolverLQNS.defaultOptions;
# MATLAB: options.keep = true;
# MATLAB: options.verbose = 1;
# MATLAB: %options.method = 'lqsim';
# MATLAB: %options.samples = 1e4;
# MATLAB: lqnssolver = SolverLQNS(model, options);
# MATLAB: AvgTableLQNS = lqnssolver.get_avg_table;
# MATLAB: AvgTableLQNS
if SolverLQNS.isAvailable():
    options = SolverLQNS.defaultOptions()
    options.keep = True
    options.verbose = 1
    # options.method = 'lqsim'
    # options.samples = int(1e4)
    lqnssolver = SolverLQNS(model, options)
    AvgTableLQNS = lqnssolver.get_avg_table()
    print('AvgTableLQNS:')
    print(AvgTableLQNS)
else:
    print("LQNS solver not available - skipping")

# this method runs the MVA solver in each layer
# MATLAB: lnoptions = SolverLN.defaultOptions;
# MATLAB: lnoptions.verbose = 0;
# MATLAB: lnoptions.seed = 2300;  
# MATLAB: options = SolverMVA.defaultOptions;
# MATLAB: options.verbose = 0;
# MATLAB: solver{1} = SolverLN(model, @(model) SolverMVA(model, options), lnoptions);
# MATLAB: AvgTable{1} = solver{1}.get_avg_table
# MATLAB: AvgTable{1}
solver = {}
AvgTable = {}

lnoptions = SolverLN.defaultOptions()
lnoptions.verbose = 0
lnoptions.seed = 2300
options = SolverMVA.defaultOptions()
options.verbose = 0
solver[1] = SolverLN(model, lambda model_arg: SolverMVA(model_arg, options), lnoptions)
AvgTable[1] = solver[1].get_avg_table()
print('AvgTable[1]:')
print(AvgTable[1])

# this method runs the NC solver in each layer
# MATLAB: lnoptions = SolverLN.defaultOptions;
# MATLAB: lnoptions.verbose = 0;
# MATLAB: lnoptions.seed = 2300;
# MATLAB: options = SolverNC.defaultOptions;
# MATLAB: options.verbose = 0;
# MATLAB: solver{2} = SolverLN(model, @(model) SolverNC(model, options), lnoptions);
# MATLAB: AvgTable{2} = solver{2}.get_avg_table
# MATLAB: AvgTable{2}
lnoptions = SolverLN.defaultOptions()
lnoptions.verbose = 0
lnoptions.seed = 2300
options = SolverNC.defaultOptions()
options.verbose = 0
solver[2] = SolverLN(model, lambda model_arg: SolverNC(model_arg, options), lnoptions)
AvgTable[2] = solver[2].get_avg_table()
print('AvgTable[2]:')
print(AvgTable[2])

# this method adapts with the features of each layer
# MATLAB: %solver{2} = SolverLN(model, @(model) LINE(model, LINE.defaultOptions), lnoptions);
# MATLAB: %AvgTable{2} = solver{2}.get_avg_table
# MATLAB: %AvgTable{2}
# solver[3] = SolverLN(model, lambda model_arg: LINE(model_arg, LINE.defaultOptions()), lnoptions)
# AvgTable[3] = solver[3].get_avg_table()
# print('AvgTable[3]:')
# print(AvgTable[3])

  Node   NodeType  QLen  Util  RespT  ResidT  ArvR  Tput
0   P1  Processor   NaN  0.25    NaN     0.0   0.0   NaN
1   P2  Processor   NaN  0.75    NaN     0.0   0.0   NaN
2   T1    RefTask  1.00  0.25    NaN     0.0   0.0  0.25
3   T2       Task  0.75  0.75    NaN     0.0   0.0  0.75
4   E1      Entry  1.00  0.25    4.0     0.0   0.0  0.25
5   E2      Entry  0.75  0.75    1.0     0.0   0.0  0.75
6   A1   Activity  1.00  0.25    4.0     0.0   0.0  0.25
7   A2   Activity  0.75  0.75    1.0     0.0   0.0  0.75
AvgTableLQNS:
  Node   NodeType  QLen  Util  RespT  ResidT  ArvR  Tput
0   P1  Processor   NaN  0.25    NaN     0.0   0.0   NaN
1   P2  Processor   NaN  0.75    NaN     0.0   0.0   NaN
2   T1    RefTask  1.00  0.25    NaN     0.0   0.0  0.25
3   T2       Task  0.75  0.75    NaN     0.0   0.0  0.75
4   E1      Entry  1.00  0.25    4.0     0.0   0.0  0.25
5   E2      Entry  0.75  0.75    1.0     0.0   0.0  0.75
6   A1   Activity  1.00  0.25    4.0     0.0   0.0  0.25
7   A2   Activity